### gathering data
- Start with a state (observation)--the one the env gives (observation_t)
- agent chooses an action based on the observation
- env executes the action and updates the internal state
- env returns results:
    - nextobs: the new state (obs) after the action
    - reward: the scalar value for that action
    - terminated: whether episode ended naturally
    - truncated: whether episode ended for some gamestopping reason (e.g., time limit)
    - done = terminated or truncated

### sampling data
- pull out a batch of samples with each being of size sequence length
- it is a dict with 5 key-val pairs
    - observation, action, reward, nextObs, done

### main training loop  
- outer loop
    - how many environment gathering sessions are we going to do, i.e., run an environment to load the buffer.
- inner loop
    - how many iterations of sampling will we take from a particular environment gathering session, i.e., how many times will we sample from a loaded buffer.
    - each time we train the world model with a sample, we then use the hidden and stochastic state to run behavior training. 

### the world model loop  
- we start with the t-1 action for the initial recurrent state (loop begins at t=1)
    - input prev recurrent state, prev latent state, and prev action (t-1 action in the data sequence)
    - outputs the recurrent state (h_t)
    - use this to input into prior net and posterior net
    - important to know that this recurrent state is used with the t encoded observation, so we essentially don't use the 0th encoded observation.
    - this means that we're only using the hidden state that has the previous action that led to the current observation to train the posterior net.
    - this also means we get 1 less in the batch length because actions are transitions from observation to observation, i.e., the transition from each t to the next one is gotten from an action (t -> t+1 -> ... -> t + batchlength-1) so there will be one less action than observations in the sequence. Hence, because we begin using observations from t, we are reducing the amount of the batch length by 1.
- after stacking each hidden states output from each t back together to get that batch length - 1, we use the concatenated hidden states with the posteriors stochastic, giving us (batchSize, batchLength-1, recurrentSize + latentLength*latentClasses). This is what is outputted by the world model to be used by the imagination rollout.

### behavior training loop (imagination rollout)  
- from the 